In [1]:
import numpy as np
import pandas as pd
path = u"./dataset/002.srt"
x = pd.read_csv(path, sep='\n')
y = np.array(x)

z = []
z.append(np.array(['1']))
for i in y:
    z.append(i)

for i in z:
    print(i)

s = ""
idx = 0
subPack = []
pack = []
for i in z:
    subPack.append(i[0])
    idx += 1
    if(idx % 3 == 0):
        pack.append(subPack)
        subPack = []
        idx = 0

start = []
end = []
for i in range(len(pack)):
    s = pack[i][1]
    s = s.replace(" --> ", "")
    s = s[0:8], s[12:20]
    a = list(s)
    pack[i][1] = a


['1']
['00:00:00,160 --> 00:00:02,510']
['核能發電到底是不是一個安全的技術呢']
['2']
['00:00:02,520 --> 00:00:03,470']
['在核災之後']
['3']
['00:00:03,480 --> 00:00:05,710']
['世界各國的核電發展又是如何呢']
['4']
['00:00:05,750 --> 00:00:07,510']
['廢核是國際趨勢嗎']
['5']
['00:00:07,600 --> 00:00:12,250']
['今天我們透過重啟核四公投來聊聊核電究竟安不安全嗎']
['6']
['00:00:23,370 --> 00:00:24,220']
['hello  大家好']
['7']
['00:00:24,230 --> 00:00:26,120']
['我是自己換回到2021公投']
['8']
['00:00:26,130 --> 00:00:26,530']
['只能哦']
['9']
['00:00:26,540 --> 00:00:27,940']
['這是這個系列的第4集']
['10']
['00:00:27,970 --> 00:00:31,000']
['那我們在前幾天已經談過公投要不要綁大選']
['11']
['00:00:31,010 --> 00:00:33,510']
['藻礁公投還有萊豬公投的3個議題']
['12']
['00:00:33,540 --> 00:00:34,800']
['那如果你還沒看完']
['13']
['00:00:34,830 --> 00:00:36,760']
['歡迎你有看完今天的影片之後呢']
['14']
['00:00:36,770 --> 00:00:38,920']
['回去看一看那今天這一集是第4集']
['15']
['00:00:38,930 --> 00:00:39,690']
['我們要來討論的']
['16']
['00:00:39,700 --> 00:00:41,880']
['是第十七案重啟核四公投']
['17']
['00:00:41,910 --> 00:00:44,820']
['不過我們發現這個議題比想像中還要複雜很多']
['18']
['00:00

In [44]:
import jieba
import jieba.analyse
import numpy as np
from bert4keras.models import build_transformer_model
from bert4keras.tokenizers import Tokenizer
from bert4keras.snippets import to_array



config_path = "./model/bert_base/chinese_L-12_H-768_A-12/bert_config.json"
checkpoint_path = './model/bert_base/chinese_L-12_H-768_A-12/bert_model.ckpt'
dict_path = './model/bert_base/chinese_L-12_H-768_A-12/vocab.txt'

model = build_transformer_model(
    config_path=config_path, checkpoint_path=checkpoint_path, with_nsp=True
)
tokenizer = Tokenizer(dict_path, do_lower_case=True)

token = []
seg = []
token_ids = ""
segment_ids = ""
result = []

for i in range(len(pack)-1):
    if (len(pack[i][2])+len(pack[i+1][2])>=512):
        pack[i][2]=pack[i][2][-128:]

    token_ids, segment_ids = tokenizer.encode(
        pack[i][2], pack[i+1][2])
    
    token_ids, segment_ids = to_array([token_ids], [segment_ids])

    probas = model.predict([token_ids, segment_ids])[0]
    print(probas)
    result.append(probas)

# token_ids = np.array([[token_ids]])
# segment_ids = np.array([[segment_ids]])
# token_ids = token_ids.reshape(1,  1, -1)
# segment_ids = segment_ids.reshape(1,  1, -1)
# probas = model.predict([token_ids, segment_ids])[0]

for i in range(len(pack)-1):
    pack[i].append(result[i][0])

idxList = []
lessThanHalf = []
lessThanHalf.append('00:00:00')
for i in range(1, len(pack)-1):
    if(pack[i][3] < 0.2):
        idxList.append(i)
        lessThanHalf.append(pack[i][1][1])

combineText = []

S = pack[0][2]
for i in range(len(pack)-1):

    token_testId, Seg_testId = tokenizer.encode(pack[i][2], pack[i+1][2])
    token_testId, Seg_testId = to_array([token_testId], [Seg_testId])
    result = model.predict([token_testId, Seg_testId])[0][0]
    # print(pack[i][2])
    if(result > 0.5):
        S += pack[i+1][2]

    else:
        S += pack[i+1][2]
        combineText.append(S)
        S = ""


wst=[]
for sentence in combineText:
    seq_list=jieba.lcut(sentence)
    wst.append(seq_list)

TF=np.zeros((np.shape(wst)[0],20))

combineStr=""
for i in combineText:
    combineStr+=i
Query=jieba.analyse.extract_tags(combineStr, topK=20, withWeight=False, allowPOS=())

for i in range(len(Query)):
    for j in range(len(wst)):
        for k in range(len(wst[j])):
            if(Query[i] == wst[j][k]):
                TF[j][i]= 1
                continue
print(TF)
keyWord=[]
deliver=[]
for i in TF:
    for j in range(len(i)):
        if(i[j]):
            deliver.append(Query[j])
    keyWord.append(deliver)
    deliver=[]

finalResult=[]
tmpProcess=[]
for i in range(len(combineText)):
    tmpProcess.append(lessThanHalf[i])
    tmpProcess.append(lessThanHalf[i+1])
    tmpProcess.append(combineText[i])
    tmpProcess.append(keyWord[i])
    finalResult.append(tmpProcess)
    tmpProcess=[]


[9.9965560e-01 3.4436595e-04]
[9.9921393e-01 7.8600773e-04]
[9.9996066e-01 3.9357445e-05]
[9.9998975e-01 1.0285766e-05]
[0.85393566 0.14606437]
[0.9546738  0.04532614]
[0.8754484  0.12455161]
[0.02091188 0.9790881 ]
[0.00359487 0.9964051 ]
[9.9997723e-01 2.2815067e-05]
[0.8936845  0.10631551]
[9.9979478e-01 2.0528778e-04]
[9.997594e-01 2.406144e-04]
[0.99884474 0.00115523]
[0.996185   0.00381493]
[0.19087882 0.80912113]
[9.9995553e-01 4.4481385e-05]
[0.02798912 0.97201085]
[0.96975625 0.03024374]
[0.97287244 0.02712756]
[0.96907234 0.0309277 ]
[9.9997103e-01 2.9000796e-05]
[9.9995530e-01 4.4671993e-05]
[0.731792   0.26820806]
[0.98755205 0.01244798]
[9.9987340e-01 1.2657508e-04]
[0.99549603 0.00450393]
[0.9838382  0.01616181]
[0.9914352  0.00856476]
[0.09723452 0.9027655 ]
[0.99694735 0.00305265]
[0.99245983 0.00754021]
[0.8336188  0.16638118]
[0.9726518  0.02734818]
[0.99394846 0.00605153]
[0.99581796 0.00418203]
[0.9886568  0.01134319]
[0.05753742 0.9424626 ]
[9.9935144e-01 6.4852240

In [46]:
finalResult

[['00:00:00',
  '00:00:26',
  '核能發電到底是不是一個安全的技術呢在核災之後世界各國的核電發展又是如何呢廢核是國際趨勢嗎今天我們透過重啟核四公投來聊聊核電究竟安不安全嗎hello  大家好我是自己換回到2021公投只能哦這是這個系列的第4集',
  ['我們', '公投', '透過', '這個', '核能', '發展', '趨勢', '核災', '核電', '核四']],
 ['00:00:26', '00:00:27', '那我們在前幾天已經談過公投要不要綁大選', ['我們', '公投', '已經']],
 ['00:00:27',
  '00:00:41',
  '藻礁公投還有萊豬公投的3個議題那如果你還沒看完歡迎你有看完今天的影片之後呢回去看一看那今天這一集是第4集我們要來討論的是第十七案重啟核四公投不過我們發現這個議題比想像中還要複雜很多',
  ['我們', '公投', '這個', '討論', '議題']],
 ['00:00:41', '00:00:47', '我們必須要先弄清楚許多關於核能還有盒子的資訊', ['我們', '核能', '必須']],
 ['00:00:47',
  '00:01:10',
  '才有辦法進入後續的討論所以這個主題呢我們會切分成上下兩集今天這一集我們會先來跟大家介紹核電廠揪竟安全過去的核災是怎麼回事也會聊到國際合的趨勢以及臺灣的核能發展現況那明天我們就會正式進入這一次核四公投的正反意見討論  當然照慣例哦在開始之前呢還是要先進一段工商服務時間臺灣第一場專門為youtube影音創作者舉辦',
  ['我們',
   '公投',
   '這個',
   '討論',
   '怎麼',
   '核能',
   '發展',
   '趨勢',
   'youtube',
   '作者',
   '核災',
   '核四']],
 ['00:01:10',
  '00:01:26',
  '年會fip即將登場嘍如果你是剛踏入youtube圈還再努力殺出重圍或者是已經有點成果正在思考變現或其他問題的創作者這場年會都會幫助你突破困境明確的知道下一步要怎麼走這種年會由臺灣新媒體影音創作者協會主辦喔',
  ['怎麼', '已經', 'youtube', '作者']],
 ['00:01:26',
  '00:0

In [10]:
# token_ids, segment_ids = tokenizer.encode(
#     pack[0][2], pack[1][2])
# token_ids, segment_ids = to_array([token_ids], [segment_ids])

# probas = model.predict([token_ids, segment_ids])[0]

In [40]:
Query=jieba.analyse.extract_tags(combineStr, topK=20, withWeight=False, allowPOS=())

In [41]:
Query

['我們',
 '公投',
 '廢料',
 '透過',
 '這個',
 '討論',
 '怎麼',
 '核能',
 '發展',
 '趨勢',
 '已經',
 'youtube',
 '核廢料',
 '低階核',
 '作者',
 '核災',
 '核電',
 '核四',
 '議題',
 '必須']

In [12]:
# for i in range(len(pack)-1):
#     if (len(pack[i][2])+len(pack[i+1][2])>=512):
#         pack[i][2]=pack[i][2][-128:]

#     token_ids, segment_ids = tokenizer.encode(
#         pack[i][2], pack[i+1][2])
    
#     token_ids, segment_ids = to_array([token_ids], [segment_ids])

#     probas = model.predict([token_ids, segment_ids])[0]
#     print(probas)
#     result.append(probas)


[9.9965560e-01 3.4436595e-04]
[9.9921393e-01 7.8600773e-04]
[9.9996066e-01 3.9357445e-05]
[9.9998975e-01 1.0285766e-05]
[0.85393566 0.14606437]
[0.9546738  0.04532614]
[0.8754484  0.12455161]
[0.02091188 0.9790881 ]
[0.00359487 0.9964051 ]
[9.9997723e-01 2.2815067e-05]
[0.8936845  0.10631551]
[9.9979478e-01 2.0528778e-04]
[9.997594e-01 2.406144e-04]
[0.99884474 0.00115523]
[0.996185   0.00381493]
[0.19087882 0.80912113]
[9.9995553e-01 4.4481385e-05]
[0.02798912 0.97201085]
[0.96975625 0.03024374]
[0.97287244 0.02712756]
[0.96907234 0.0309277 ]
[9.9997103e-01 2.9000796e-05]
[9.9995530e-01 4.4671993e-05]
[0.731792   0.26820806]
[0.98755205 0.01244798]
[9.9987340e-01 1.2657508e-04]
[0.99549603 0.00450393]
[0.9838382  0.01616181]
[0.9914352  0.00856476]
[0.09723452 0.9027655 ]
[0.99694735 0.00305265]
[0.99245983 0.00754021]
[0.8336188  0.16638118]
[0.9726518  0.02734818]
[0.99394846 0.00605153]
[0.99581796 0.00418203]
[0.9886568  0.01134319]
[0.05753742 0.9424626 ]
[9.9935144e-01 6.4852240

array([[ 101, 6917, 1139, 6158, 5050,  868, 3221,  856, 7389, 3417, 2450,
        3160,  102,  891, 1963, 1008, 3221, 3417, 7442, 2449, 1058,  886,
        4500, 6882, 4638, 6132, 4289, 1557,  102]])

In [16]:
# token_ids = np.array([[token_ids]])
# segment_ids = np.array([[segment_ids]])
# token_ids = token_ids.reshape(1,  1, -1)
# segment_ids = segment_ids.reshape(1,  1, -1)



# probas = model.predict([token_ids, segment_ids])[0]


# token_ids, segment_ids = tokenizer.encode(
#     pack[0][2],
#     pack[1][2])
# token_ids, segment_ids = to_array([token_ids], [segment_ids])
# token_ids.shape
# probas = model.predict([token_ids, segment_ids])[0]
# probas



# for i in range(len(pack)-1):
#     pack[i].append(result[i][0])



# idxList = []
# lessThanHalf = []
# lessThanHalf.append('00:00:00')
# for i in range(1, len(pack)-1):
#     if(pack[i][3] < 0.2):
#         idxList.append(i)
#         lessThanHalf.append(pack[i][1][1])

# combineText = []

# S = pack[0][2]
# for i in range(len(pack)-1):

#     token_testId, Seg_testId = tokenizer.encode(pack[i][2], pack[i+1][2])
#     token_testId, Seg_testId = to_array([token_testId], [Seg_testId])
#     result = model.predict([token_testId, Seg_testId])[0][0]
#     # print(pack[i][2])
#     if(result > 0.5):
#         S += pack[i+1][2]

#     else:
#         S += pack[i+1][2]
#         combineText.append(S)
#         S = ""

# wst=[]

# wst = ws(
#     combineText,
# )
# del ws


# for sentence in combineText:
#     seq_list=jieba.lcut(sentence)
#     wst.append(seq_list)

# seq_list = ws(
#     combineText,
#      sentence_segmentation = True, # To consider delimiters
#      segment_delimiter_set = {",", "。", ":", "?", "!", ";"}), # This is the defualt set of delimiters
#     # recommend_dictionary = dictionary1, # words in this dictionary are encouraged
#     # coerce_dictionary = dictionary2, # words in this dictionary are forced

# TF=np.zeros((np.shape(wst)[0],20))



# for i in range(len(Query)):
#     for j in range(len(wst)):
#         for k in range(len(wst[j])):
#             if(Query[i] == wst[j][k]):
#                 TF[j][i]= 1
#                 continue

# keyWord=[]
# deliver=[]
# for i in TF:
#     for j in range(len(i)):
#         if(i[j]):
#             deliver.append(Query[j])
#     keyWord.append(deliver)
#     deliver=[]
# finalResult=[]
# tmpProcess=[]
# time=[]

# for i in range(len(combineText)):
#     tmpProcess.append(lessThanHalf[i])
#     tmpProcess.append(lessThanHalf[i+1])
#     tmpProcess.append(combineText[i])
#     tmpProcess.append(keyWord[i])
#     finalResult.append(tmpProcess)
#     tmpProcess=[]

IndexError: invalid index to scalar variable.

In [14]:
TF

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.]])

In [36]:
# S = combineText[0]
# combineText_1=[]
# for i in range(len(combineText)-1):

#     token_testId, Seg_testId = tokenizer.encode(combineText[i], combineText[i+1])
#     token_testId, Seg_testId = to_array([token_testId], [Seg_testId])
#     result = (model.predict([token_testId, Seg_testId])[0][0])
#     print(result)
    # if(result > 0.01):
    #     S += combineText[i+1]

    # else:
    #     S += combineText[i+1]
    #     print(S)
    #     combineText_1.append(S)

0.9999825
0.9999914
0.9999753
0.9999875
0.99999404
0.99997735
0.99995375
0.9999602


In [6]:
for i in range(len(Query)):
    for j in range(len(wst)):
        for k in range(len(wst[j])):
            if(Query[i] == wst[j][k]):
                TF[j][i]= 1
                continue

In [8]:
wst

[['核能',
  '發電',
  '到底',
  '是不是',
  '一個',
  '安全',
  '的',
  '技術',
  '呢',
  '在',
  '核災',
  '之',
  '後',
  '世界',
  '各國',
  '的',
  '核電',
  '發展',
  '又',
  '是',
  '如何',
  '呢',
  '廢核',
  '是',
  '國際',
  '趨勢',
  '嗎',
  '今天',
  '我們',
  '透過',
  '重啟',
  '核四',
  '公投',
  '來',
  '聊聊',
  '核電',
  '究竟',
  '安',
  '不',
  '安全',
  '嗎',
  'hello',
  ' ',
  ' ',
  '大家',
  '好',
  '我',
  '是',
  '自己',
  '換',
  '回到',
  '2021',
  '公投',
  '只能',
  '哦',
  '這是',
  '這個',
  '系列',
  '的',
  '第',
  '4',
  '集'],
 ['那', '我們', '在', '前', '幾天', '已經', '談過', '公投', '要', '不要', '綁大選'],
 ['藻礁',
  '公投',
  '還有萊豬',
  '公投',
  '的',
  '3',
  '個',
  '議題',
  '那',
  '如果',
  '你',
  '還沒看',
  '完歡',
  '迎',
  '你',
  '有',
  '看',
  '完',
  '今天',
  '的',
  '影片',
  '之',
  '後',
  '呢',
  '回去',
  '看一看',
  '那',
  '今天',
  '這',
  '一集',
  '是',
  '第',
  '4',
  '集',
  '我們',
  '要',
  '來',
  '討論',
  '的',
  '是',
  '第十七',
  '案重',
  '啟核',
  '四',
  '公投',
  '不過',
  '我們',
  '發現',
  '這個',
  '議題',
  '比',
  '想像中',
  '還要',
  '複',
  '雜',
  '很多'],
 ['我們', '必須', '要', '先', '弄清楚',

In [58]:
# dataString = []
# for i in range(len(idxList)):
#     dataString.append(pack[i][2])


In [35]:
Str = []
a = ''
b = 0
k = 0
for i in range(len(idxList)):
    if(i == idxList[k]):
        for j in range(b, i+1):
            a += pack[j][2]
        Str.append(a)
        k += 1
        a = ''
        b = i+1


In [36]:
Str

['核能發電到底是不是一個安全的技術呢在核災之後世界各國的核電發展又是如何呢廢核是國際趨勢嗎今天我們透過重啟核四公投來聊聊核電究竟安不安全嗎hello  大家好我是自己換回到2021公投只能哦',
 '這是這個系列的第4集']

In [37]:
combineText

['核能發電到底是不是一個安全的技術呢在核災之後世界各國的核電發展又是如何呢廢核是國際趨勢嗎今天我們透過重啟核四公投來聊聊核電究竟安不安全嗎hello  大家好我是自己換回到2021公投只能哦這是這個系列的第4集',
 '那我們在前幾天已經談過公投要不要綁大選',
 '藻礁公投還有萊豬公投的3個議題那如果你還沒看完歡迎你有看完今天的影片之後呢回去看一看那今天這一集是第4集我們要來討論的是第十七案重啟核四公投不過我們發現這個議題比想像中還要複雜很多',
 '我們必須要先弄清楚許多關於核能還有盒子的資訊',
 '才有辦法進入後續的討論所以這個主題呢我們會切分成上下兩集今天這一集我們會先來跟大家介紹核電廠揪竟安全過去的核災是怎麼回事也會聊到國際合的趨勢以及臺灣的核能發展現況那明天我們就會正式進入這一次核四公投的正反意見討論  當然照慣例哦在開始之前呢還是要先進一段工商服務時間臺灣第一場專門為youtube影音創作者舉辦',
 '年會fip即將登場嘍如果你是剛踏入youtube圈還再努力殺出重圍或者是已經有點成果正在思考變現或其他問題的創作者這場年會都會幫助你突破困境明確的知道下一步要怎麼走這種年會由臺灣新媒體影音創作者協會主辦喔',
 '串聯了多位頂尖創作者一起分享他們在內容策略個人發展已經趨勢以及商業洞察的經驗和觀點除此之外呢',
 '協會還邀請到youtube官方代表和ip為大家提供最準確的平臺動向和建議那如果你還沒創作者但正在準備入行或者是有興趣了解當然也能參加  而像年會呢將在2022年的1月10日登場除了現場席位呢也有線上影片票可以讓大家遠端參與目前優惠票種真的熱烈的範疇當中那因為數量有限所以有幸確認請點擊下方的諮詢連接了解更詳細的活動內容後速速報名他要討論合適',
 '我們必須要先了解一下核能電廠是怎麼運作的核能發電廠的發力原理呢是透過引發核分裂連鎖反應釋放的龐大能量來燒水再透過發電機把這個能量轉化成電力最後再透過冷卻水把多餘的廢熱排掉那目前世界上面多數的核電廠使用的燃料棒是濃度三到5倍爾見的鈾235來發電而經過長時間的反應之後呢燃料棒就會變成帶有放射性的核廢料那這些核廢料該怎麼處理呢一般來說核廢料會依照放射性的強度區分成高階核廢料還有低階核廢料兩大類高階核廢料指的是用過的燃料棒而低階核廢料通常指的是在輻射場所使用產生的廢棄物而這些廢棄物呢不一定都會被輻射污染

In [38]:
combineStr=""
for i in combineText:
    combineStr+=i

In [39]:
combineStr

'核能發電到底是不是一個安全的技術呢在核災之後世界各國的核電發展又是如何呢廢核是國際趨勢嗎今天我們透過重啟核四公投來聊聊核電究竟安不安全嗎hello  大家好我是自己換回到2021公投只能哦這是這個系列的第4集那我們在前幾天已經談過公投要不要綁大選藻礁公投還有萊豬公投的3個議題那如果你還沒看完歡迎你有看完今天的影片之後呢回去看一看那今天這一集是第4集我們要來討論的是第十七案重啟核四公投不過我們發現這個議題比想像中還要複雜很多我們必須要先弄清楚許多關於核能還有盒子的資訊才有辦法進入後續的討論所以這個主題呢我們會切分成上下兩集今天這一集我們會先來跟大家介紹核電廠揪竟安全過去的核災是怎麼回事也會聊到國際合的趨勢以及臺灣的核能發展現況那明天我們就會正式進入這一次核四公投的正反意見討論  當然照慣例哦在開始之前呢還是要先進一段工商服務時間臺灣第一場專門為youtube影音創作者舉辦年會fip即將登場嘍如果你是剛踏入youtube圈還再努力殺出重圍或者是已經有點成果正在思考變現或其他問題的創作者這場年會都會幫助你突破困境明確的知道下一步要怎麼走這種年會由臺灣新媒體影音創作者協會主辦喔串聯了多位頂尖創作者一起分享他們在內容策略個人發展已經趨勢以及商業洞察的經驗和觀點除此之外呢協會還邀請到youtube官方代表和ip為大家提供最準確的平臺動向和建議那如果你還沒創作者但正在準備入行或者是有興趣了解當然也能參加  而像年會呢將在2022年的1月10日登場除了現場席位呢也有線上影片票可以讓大家遠端參與目前優惠票種真的熱烈的範疇當中那因為數量有限所以有幸確認請點擊下方的諮詢連接了解更詳細的活動內容後速速報名他要討論合適我們必須要先了解一下核能電廠是怎麼運作的核能發電廠的發力原理呢是透過引發核分裂連鎖反應釋放的龐大能量來燒水再透過發電機把這個能量轉化成電力最後再透過冷卻水把多餘的廢熱排掉那目前世界上面多數的核電廠使用的燃料棒是濃度三到5倍爾見的鈾235來發電而經過長時間的反應之後呢燃料棒就會變成帶有放射性的核廢料那這些核廢料該怎麼處理呢一般來說核廢料會依照放射性的強度區分成高階核廢料還有低階核廢料兩大類高階核廢料指的是用過的燃料棒而低階核廢料通常指的是在輻射場所使用產生的廢棄物而這些廢棄物呢不一定都會被輻射污染但安全起降全部呢還出被算作是低階核廢料'